## Exercice #1 : Trouve de nouveaux ami.e.s !

### Description du problème

Le but de l'exercice est d'implémenter la fonctionnalité permettant d'obtenir les ami.e.s communs de deux utilisateurs sur un réseau social tel que Facebook, dans un cadre MapReduce. 

La notion d'ami dans un réseau social est en général bidirectionnelle (si `A` est un ami de `B` alors `B` est un ami de `A`) et on se limite ici à la recherche des amis communs de deux utilisateurs qui sont déjà amis. Cela pourra par exemple servir à proposer dans un deuxième temps à l'utilisateur `A` des amis de `B` qu'ils n'ont pas encore en commun (et réciproquement).

L'approche classique la plus directe pour trouver les amis communs de deux utilisateurs est de calculer l'intersection des listes d'amis des deux utilisateurs, ce qui demande de rassembler ces listes au même endroit (i.e. sur une même machine) pour pouvoir effectuer le traitement. Pour chaque utilisateur du réseau social, il faudrait faire cela avec chacun de ses amis et relancer le calcul régulièrement car les listes d'amis évoluent rapidement. Dans le cas d'un très gros réseau social tel que Facebook, dont les données sont réparties sur de multiples serveurs, la localisation et les transferts des données que cela implique seraient très coûteux et les temps de réponse pour obtenir l'ensemble des calculs seraient prohibitifs.


Le but est donc ici de définir des traitements appliqués aux données de chaque utilisateur qui soient indépendants les uns des autres (donc distribuables ou parallélisables), et dont les résultats seront rassemblés ensuite pour pouvoir calculer les intersections de listes d'amis de chaque paires d'utilisateurs amis. Ce type de flux de traitements correspond exactement à celui du modèle MapReduce que nous allons donc utilisé pour l'implémentation.

Le principe est expliqué sur l'exemple suivant :

- soit `A` un utilisateur et `[B, C, D, E]` sa liste d'amis;
- soit `B` un autre utilisateur (ami de `A`) et `[A, C, E]` sa liste d'amis;

La liste d'amis communs à `A` et `B` pourra être obtenue en faisant l'intersection entre les listes `[B, C, D, E]` et `[A, C, E]`. Le principe est de mettre en relation ces deux listes en les associant à une même clé, ici le couple d'utilisateur `(A, B)`. 

Dans le cadre MapReduce, les données d'un utilisateur `A` (l'utilisateur lui même, associé à sa liste d'amis) seront traitées par un même `mapper` qui en sortie associera donc la liste des amis de `A` à chaque couple `(A, *X*)` (ou `(*X*, A)` suivant les cas, cf remarque ci-après), avec `*X*` un des amis de `A`. Ces couples `(A, *X*)` constitueront donc des clés intermédiaires du flux de traitement MapReduce (en sortie des `mapper`) et la liste d'amis de `A` associée constituera une valeur intermédiaire.

Pour l'exemple donné ci-dessus cela donnerait pour le `mapper` s'occupant des données de `A` :
- `(A, B)` --> `[B, C, D, E]`
- `(A, C)` --> `[B, C, D, E]`
- `(A, D)` --> `[B, C, D, E]`
- `(A, E)` --> `[B, C, D, E]`

Et pour le `mapper` s'occupant des données de `B` :
- `(A, B)` --> `[A, C, E]`
- `(B, C)` --> `[A, C, E]`
- `(B, E)` --> `[A, C, E]`

**Remarque** : notez que les couples d'utilisateurs constituant les clés intermédiaires (en sorties des `mapper`) devront systématiquement être triés (par ordre alphabétique) afin d'obtenir une même clé `(A, B)` pour le mapper s'occupant de `A` et pour le `mapper` s'occupant de son ami `B` (et non pas `(A, B)` dans un cas et `(B, A)` dans l'autre, ce qui constituerait deux clés différentes).

La partie `partitionner` de MapReduce se chargera ensuite de rassembler les listes d'utilisateurs associées à une même clé intermédiare et les `reducer` calculeront pour finir l'intersection des listes associées à chaque clé intermédiaire. Dans notre cas, il y aura exactement deux listes rassemblées par clé intermédiaire au niveau du `partitionner` : pour une clé `(A, B)` le `partitionner` rassemblera la liste d'amis de `A` et la liste d'amis de `B`. 

### Mise en oeuvre

La fonction `map_reduce()` écrite lors du TD sur MapReduce est importée dans la cellule ci-dessous depuis le fichier `map_reduce.py` fourni (qui contient également la fonction `partitionner()`).

Le fichier `friends.txt` donne sur chaque ligne le nom d'un utilisateur (premier nom) suivi de la liste de ses ami.e.s, le tout séparé par des espaces. La fonction `read_friends()` donnée ci-dessous permet de récupérer ces informations sous forme d'un dictionnaire avec comme clés les utilisateurs et comme valeurs les listes d'ami.e.s associée. Cette fonction prendra en paramètre le nom du fichier contenant les utilisateurs et leur liste d'ami.e.s.

In [14]:
from map_reduce import map_reduce

def read_friends(filename):
    friends = {}
    with open(filename, 'rt', encoding='utf8') as ifile :
        for line in ifile.readlines():
            all = line.strip().split()
            friends[all[0]] = all[1:]
    return friends

friends = read_friends("friends.txt")
print(friends)

{'Therese': ['Vincent', 'Loevane', 'Rim', 'Sarra', 'Pauline', 'Oriane', 'Bleuenn', 'Emilien', 'Clemence', 'Clement', 'Mathis'], 'Sarra': ['Elisa', 'Clara_m', 'Mathis', 'Annaelle', 'Vincent', 'Sterenn', 'Loevane', 'Rim', 'Therese', 'Maelys', 'Oriane', 'Bleuenn'], 'Marie': ['Marion', 'Hugo', 'Clara_f', 'Bleuenn', 'Clemence', 'Jules', 'Annaelle', 'Sterenn', 'Loevane'], 'Elisa': ['Lucie', 'Clement', 'Jules', 'Clara_m', 'Sterenn', 'Floriane', 'Guillemette', 'Rim', 'Sarra'], 'Marion': ['Clemence', 'Floriane', 'Guillemette', 'Fatima_ezzahrae', 'Marie', 'Sterenn', 'Loevane', 'Anatole', 'Nicolas', 'Margaux', 'Maelys', 'Lucie', 'Pauline', 'Bleuenn'], 'Oriane': ['Sarra', 'Clara_f', 'Therese', 'Maelys', 'Bleuenn', 'Clemence', 'Clara_m', 'Fatima_ezzahrae', 'Annaelle', 'Floriane', 'Anatole', 'Rim'], 'Julianne': ['Mathis', 'Vincent', 'Rim', 'Nicolas', 'Sterenn', 'Maelys', 'Bleuenn'], 'Bleuenn': ['Marion', 'Marie', 'Sarra', 'Hugo', 'Therese', 'Lucie', 'Oriane', 'Clemence', 'Annaelle', 'Julianne', 'Fat

#### Partie `mapper`

**Question 1.1**

Ecrire une fonction `mapper_friends()` qui prendra en entrée un tuple `(clé d'entrée, valeur d'entrée)` avec comme clé le nom d'un utilisateur, et comme valeur la liste des amis de cet utilisateur, et qui renverra une liste de tuples `(clé ìntermédiaire, valeur intermédiaire)` avec comme clés les tuples `(utilisateur, ami)` (pour chaque ami de l'utilisateur) et comme valeur la liste des amis de l'utilisateur. Les tuples `(utilisateur, ami)` fournis comme clé intermédiaires en sortie du mapper devront systématiquement être triés par ordre alphabétique.

Tester le résultat obtenu pour les exemples des utilisateur `A` et `B` (et leur liste d'ami.e.s repectives) donnés ci-dessus.

In [15]:
def mapper_friends(cle_val):
    user, friends = cle_val
    return [(tuple(sorted((user,f))), friends) for f in friends]

print(mapper_friends(("A", ["B", "C", "D", "E"])))
print(mapper_friends(("B", ["A", "C", "E"])))

[(('A', 'B'), ['B', 'C', 'D', 'E']), (('A', 'C'), ['B', 'C', 'D', 'E']), (('A', 'D'), ['B', 'C', 'D', 'E']), (('A', 'E'), ['B', 'C', 'D', 'E'])]
[(('A', 'B'), ['A', 'C', 'E']), (('B', 'C'), ['A', 'C', 'E']), (('B', 'E'), ['A', 'C', 'E'])]


#### Partie `reducer`

On souhaite écrire un `reducer` générique qui permette de calculer l'intersection de plusieurs listes (deux listes ou plus) associées à une clé.

Le reducer prendra en entrée un tuple `(clé intermédiaire, [liste de valeurs intermédiaires])` avec dans notre cas, comme clé intermédiaire un tuple `(utilisateur1, utilisateur2)` et comme liste de valeurs intermédiaires une liste de deux listes : la liste d'amis de l'`utilisateur1` et la liste d'amis de l'`utilisateur2`.

**Question 1.2**

Ecrire une fonction `interset_2_lists()` qui prend en entrée deux listes et renvoie en sortie une liste résultat de l'intersection des deux listes d'entrée.

Tester sur les listes `l1` et `l2` fournies ci-dessous.

In [16]:
def intersect_2_lists(l1, l2):
    return list(set.intersection(set(l1), set(l2)))

l1 = ['a', 'b', 'c', 'd', 'e', 'f']
l2 = ['z', 'd', 'm', 'a', 'c']

intersect_2_lists(l1, l2)

['a', 'c', 'd']

**Question 1.3**

Ecrire une instruction utilisant la fonction `reduce()` du module `functools` et la fonction `interset_2_lists()` permettant de calculer l'intersection d'un nombre quelconque de listes.

Tester avec l'intersection des quatre listes `l1`, `l2` (fournies ci-dessus), `l3` et `l4` (fournies ci-dessous).

In [17]:
from functools import reduce
l3 = ['o', 'd', 'a', 'r']
l4 = ['d', 't', 'x', 'a', 'c']

reduce(intersect_2_lists, [l1, l2, l3, l4])

['a', 'd']

**Question 1.4**

Ecrire une fonction `reducer_intersect()` qui prend en entrée un tuple `(clé , [liste de listes])` et qui renvoie en sortie un tuple `(clé de sortie, liste de sortie)` avec la clé de sortie égale à la clé d'entrée et la liste de sortie correspondant à l'intersection des listes de la liste d'entrée (deuxième élément du tuple d'entrée).

Tester sur la liste des listes `l1`, `l2`, `l3` et `l4` fournies ci-dessus.

In [18]:
def reducer_intersect(cle_val):
    cle, list_of_lists = cle_val
    return [(cle, list(reduce(intersect_2_lists,list_of_lists)))]

reducer_intersect((('u1', 'u2'), [l1, l2, l3, l4]))

[(('u1', 'u2'), ['a', 'd'])]

**Question 1.5**

Utiliser les fonctions `map_reduce()`, `mapper_friend()` et `reducer_intersect()` pour afficher les listes d'amis communs de chaque paire d'ami.e.s du fichier `friends.txt` comme dans l'exemple ci-dessous (début de l'affichage) :

`('Anatole', 'Clement') --> ['Emilien', 'Vincent', 'Maelys']`<br>
`('Anatole', 'Emilien') --> ['Clement', 'Jules']`<br>
`('Anatole', 'Jules') --> ['Emilien', 'Maelys']`<br>
`('Anatole', 'Lucie') --> ['Marion', 'Vincent']`<br>
`('Anatole', 'Maelys') --> ['Marion', 'Oriane', 'Jules', 'Clement', 'Mathis']`<br>
`...`

Sur chaque ligne la paire d'ami.e.s est à gauche de la flèche `-->` et la liste des ami.e.s communs de ces deux ami.e.s est à droite de la flèche. Les résultats seront affichés triés par ordre alphabétique des paires d'ami.e.s.

In [19]:
for pair, frs in sorted(map_reduce(friends.items(), mapper_friends, reducer_intersect)):
    print(f"{pair} --> {frs}")

('Anatole', 'Clement') --> ['Emilien', 'Vincent', 'Maelys']
('Anatole', 'Emilien') --> ['Clement', 'Jules']
('Anatole', 'Jules') --> ['Emilien', 'Maelys']
('Anatole', 'Lucie') --> ['Marion', 'Vincent']
('Anatole', 'Maelys') --> ['Marion', 'Oriane', 'Jules', 'Clement', 'Mathis']
('Anatole', 'Marion') --> ['Lucie', 'Maelys']
('Anatole', 'Mathis') --> ['Maelys']
('Anatole', 'Oriane') --> ['Maelys']
('Anatole', 'Vincent') --> ['Clement', 'Lucie']
('Annaelle', 'Bleuenn') --> ['Oriane', 'Marie', 'Sarra']
('Annaelle', 'Jules') --> ['Maelys', 'Marie']
('Annaelle', 'Maelys') --> ['Oriane', 'Jules', 'Sarra', 'Mathis']
('Annaelle', 'Marie') --> ['Sterenn', 'Jules', 'Bleuenn']
('Annaelle', 'Mathis') --> ['Maelys', 'Sarra']
('Annaelle', 'Oriane') --> ['Maelys', 'Sarra', 'Bleuenn']
('Annaelle', 'Sarra') --> ['Oriane', 'Maelys', 'Sterenn', 'Vincent', 'Bleuenn', 'Mathis']
('Annaelle', 'Sterenn') --> ['Vincent', 'Marie', 'Sarra']
('Annaelle', 'Vincent') --> ['Sterenn', 'Sarra']
('Bleuenn', 'Clemence') 